In [1]:
# IMPORTS FOR ALL THE PROJECT
import numpy as np
import os
import requests
from numpy.random import default_rng
import matplotlib
import matplotlib.pyplot as plt
import tarfile
from six.moves import urllib
import pandas as pd



In [2]:
# PATH TO THE FILES

urls= ['https://raw.githubusercontent.com/Plexiglace-EL/projet-IA/main/general_data.csv',
       'https://raw.githubusercontent.com/Plexiglace-EL/projet-IA/main/manager_survey_data.csv',
       'https://raw.githubusercontent.com/Plexiglace-EL/projet-IA/main/employee_survey_data.csv',
       'https://raw.githubusercontent.com/Plexiglace-EL/projet-IA/main/in_time.csv',
       'https://raw.githubusercontent.com/Plexiglace-EL/projet-IA/main/out_time.csv']

paths = ['./datasets/general_data.csv',
         './datasets/manager_survey_data.csv',
         './datasets/employee_survey_data.csv',
         './datasets/in_time.csv',
         './datasets/out_time.csv']

In [3]:

def fetch_datas(url, path):
    localfile, header = urllib.request.urlretrieve(url, path)
    print(localfile)
    return localfile

def extractZip(path):
    with tarfile.open(path) as tf:
        tf.extractall('./datasets')


In [4]:
def import_all_files():
    print('==='*4, "BEGIN", '==='*4)


    if not os.path.exists('./datasets/'):
        print('==='*4, "CREATING \"./datasets/\" DIRECTORY", '==='*4)
        os.mkdir('./datasets')

    for i, url in enumerate(urls):
        fetch_datas(urls[i], paths[i])

    print('==='*4, "FINISHED", '==='*4)

In [5]:
def load_datas(path):
    datas = pd.read_csv(path)
    return datas

In [6]:
# FUNCTION TO DOWNLOAD THE FILES FROM ONLINE REPO TO LOCAL FOLDER

import_all_files()

============ BEGIN ============
./datasets/general_data.csv
./datasets/manager_survey_data.csv
./datasets/employee_survey_data.csv
./datasets/in_time.csv
./datasets/out_time.csv
============ FINISHED ============


In [6]:
# IMPORTING DATA IN DATAFRAME VARIABLES

general = load_datas(paths[0])
manager_survey = load_datas(paths[1])
employee_survey = load_datas(paths[2])
in_time = load_datas(paths[3])
out_time = load_datas(paths[4])

In [7]:
def converting_string_to_dates(in_time, out_time):
    # RENAMING THE UNNAMED COLUMN (ID_EMPLOYEE) FOR IT TO BE USEABLE MORE EASILY IN THE FUTURE

    in_time = in_time.rename(columns={'Unnamed: 0': 'id_employee'})
    out_time = out_time.rename(columns={'Unnamed: 0': 'id_employee'})


    # WE CAN USE THE COLUMNS OF ONLY ONE FILE BECAUSE THEY ARE BOTH THE SAME AND CONTAINS THE SAME HEADERS NAME
    # IT HAS BEEN VERIFIED USING LIST COMPARISON AND UNIQUE() CONDITION TO FILTER AND WE END WITH A TOTAL OF 262 COLUMNS
    cols = in_time.columns


    # CONVERTING THE DATES TO REAL PANDAS DATES OBJECT
    for i in cols:
        if i != 'id_employee':
            in_time[i] = pd.to_datetime(in_time[i])
            out_time[i] = pd.to_datetime(out_time[i])

    return in_time, out_time

In [8]:
#converting in and out time dataframes into pandas date obj
in_time, out_time = converting_string_to_dates(in_time, out_time)

In [9]:
# using the converted dates to calculate the working time of each employee
def calculate_working_time():

    working_time = in_time.copy()
    for i in working_time:
        if i != 'id_employee':
            working_time[i] = out_time[i] - working_time[i]
            
    return working_time

In [10]:
working_time = calculate_working_time()


In [11]:
# Encoder to convert the text data to actual values, it returns a corresponding matrix and the converted column

from sklearn.preprocessing import LabelEncoder
label_enc = LabelEncoder()

def personnal_encoder(df, col):
    df[col] = label_enc.fit_transform(df[col])
    labels = label_enc.classes_.copy()
    for index, l in enumerate(labels):
        labels[index] = [index, l]

    return labels, df



In [12]:
businesstravel_labels, general = personnal_encoder(general, 'BusinessTravel')
attrition_labels, general = personnal_encoder(general, 'Attrition')
education_labels, general = personnal_encoder(general, 'EducationField')
department_labels, general = personnal_encoder(general, 'Department')
gender_labels, general = personnal_encoder(general, 'Gender')
ovr18_labels, general = personnal_encoder(general, 'Over18')


In [20]:
general.describe()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,mean,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
count,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4410.0,4410.000000,4410.000000,...,4410.000000,4410.000000,4410.000000,4410.000000,4410.000000,4385.000000,4390.000000,4372.000000,4410.000000,4410.000000
mean,36.923810,0.161224,1.607483,1.260544,9.192517,2.912925,2.247619,1.0,2205.500000,0.600000,...,2.799320,7.008163,2.187755,4.123129,7.700830,2.723603,2.728246,2.761436,2.729932,3.153741
std,9.133301,0.367780,0.665304,0.527673,8.105026,1.023933,1.331067,0.0,1273.201673,0.489954,...,1.288978,6.125135,3.221699,3.567327,1.340277,1.092756,1.101253,0.706245,0.711400,0.360742
min,18.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,5.950000,1.000000,1.000000,1.000000,1.000000,3.000000
25%,30.000000,0.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.0,1103.250000,0.000000,...,2.000000,3.000000,0.000000,2.000000,6.670000,2.000000,2.000000,2.000000,2.000000,3.000000
50%,36.000000,0.000000,2.000000,1.000000,7.000000,3.000000,2.000000,1.0,2205.500000,1.000000,...,3.000000,5.000000,1.000000,3.000000,7.410000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,43.000000,0.000000,2.000000,2.000000,14.000000,4.000000,3.000000,1.0,3307.750000,1.000000,...,3.000000,9.000000,3.000000,7.000000,8.370000,4.000000,4.000000,3.000000,3.000000,3.000000
max,60.000000,1.000000,2.000000,2.000000,29.000000,5.000000,5.000000,1.0,4410.000000,1.000000,...,6.000000,40.000000,15.000000,17.000000,11.030000,4.000000,4.000000,4.000000,4.000000,4.000000


In [21]:
# Adding the calculated mean value of the working time of each employee to the main dataframe
mean_time = working_time.copy()
mean_time = mean_time.drop('id_employee', axis=1)
general['mean'] = mean_time.mean(axis = 1)
general['mean'] = general['mean'].dt.total_seconds() / 3600
general['mean'] = round(general['mean'], 2)
# Adding the rest of the datas (from the survey)
general['EnvironmentSatisfaction'] = employee_survey['EnvironmentSatisfaction']
general['JobSatisfaction'] = employee_survey['JobSatisfaction']
general['WorkLifeBalance'] = employee_survey['WorkLifeBalance']
general['JobInvolvement'] = manager_survey['JobInvolvement']
general['PerformanceRating'] = manager_survey['PerformanceRating']

# Reformating the column of the existing general DF for comprehension
general = general.reindex(columns=['EmployeeID','Age','Attrition','BusinessTravel','Department','DistanceFromHome','Education','EducationField','EmployeeCount','Gender','JobLevel','JobRole','MaritalStatus','MonthlyIncome','NumCompaniesWorked','Over18','PercentSalaryHike','StandardHours','StockOptionLevel','TotalWorkingYears','TrainingTimesLastYear','YearsAtCompany','YearsSinceLastPromotion','YearsWithCurrManager','mean','EnvironmentSatisfaction','JobSatisfaction','WorkLifeBalance','JobInvolvement','PerformanceRating'])
general

,EmployeeID,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,mean,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,51,0,2,2,6,2,1,1,0,...,6,1,0,0,7.37,3.0,4.0,2.0,3,3
1,2,31,1,1,1,10,1,1,1,0,...,3,5,1,4,7.72,3.0,2.0,4.0,2,4
2,3,32,0,1,1,17,4,4,1,1,...,2,5,0,3,7.01,2.0,2.0,1.0,3,3
3,4,38,0,0,1,2,5,1,1,1,...,5,8,7,5,7.19,4.0,4.0,3.0,2,3
4,5,32,0,2,1,10,1,3,1,1,...,2,6,0,4,8.01,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,42,0,2,1,5,4,3,1,0,...,5,3,0,2,8.52,4.0,1.0,3.0,3,3
4406,4407,29,0,2,1,2,4,3,1,1,...,2,3,0,2,6.09,4.0,4.0,3.0,2,3
4407,4408,25,0,2,1,25,2,1,1,1,...,4,4,1,2,7.71,1.0,3.0,3.0,3,4
4408,4409,42,0,2,2,18,2,3,1,1,...,2,9,7,8,9.49,4.0,1.0,3.0,2,3
